## Aurora v 1.1 - Estável

In [39]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer


# v 1.1 - Cross-Validation = 34.11 / Previsão Final = 40.20

# Carregamento dos dados
data_treino = pd.read_csv('/home/caio/Estudo/kaggle_02/train.csv')
data_prev = pd.read_csv('/home/caio/Estudo/kaggle_02/test.csv')

class Aurora:

    def __init__(self, data_treino, data_previsao):
        self.data_treino = data_treino                                                             # Carga DataFrame Treino
        self.data_treino = self.data_treino.dropna(axis = 0, subset=['Price'], inplace=True)
        self.data_previsao = data_previsao                                                         # Carga DataFrame Previsão (Previsão = feature)
        self.target, self.feature_categorical, self.feature_numerical, self.previsao_categorical, self.previsao_numerical = self.splitter(data_treino, data_previsao)
        
        
        
        # Tratemento Categórico
        self.feature_categorical_encoded = self.one_hot(self.feature_categorical)
        self.previsao_categorical_encoded = self.one_hot(self.previsao_categorical)
        
        # Juntando data Categorica(tratada) e data numérica
        self.feature_completed = self.fusion(self.feature_categorical_encoded, self.feature_numerical)
        self.previsao_completed = self.fusion(self.previsao_categorical_encoded, self.previsao_numerical)


        # Treinando Modelo
        self.treino_final = self.treino(self.feature_completed, self.target)

        # Fazendo Previsão
        self.prev = self.previsao(self.previsao_completed)

        # Salvando Arquivo
        self.impressao(self.prev)

        print('Aurora Finalizada Com Sucesso!')


    def splitter(self, data_treino, data_previsao):
        feature = data_treino.drop(columns=['Price'])
        target = data_treino['Price']
        feature_categorical = feature.select_dtypes(include=['object', 'category'])
        feature_numerical = feature.select_dtypes(include=['int64', 'float64'])
        previsao_categorical = data_previsao.select_dtypes(include=['object', 'category'])
        previsao_numerical = data_previsao.select_dtypes(include=['int64', 'float64'])

        return target, feature_categorical, feature_numerical, previsao_categorical, previsao_numerical

    def one_hot(self, data):
        return pd.get_dummies(data)

    def fusion(self, data1, data2):
        return data1.join(data2, how='left')

    def treino(self, feature, target):
        self.model = XGBRegressor()
        scores = -1 * cross_val_score(self.model, feature, target,
                              cv=5,
                              scoring='neg_mean_absolute_error')

        self.model.fit(feature, target)

        print("Average MAE score:")
        print(scores.mean())

    def previsao(self, data):
        if self.model is None:
            raise ValueError("O Modelo Não Foi Treinado!")

        previsao = self.model.predict(data)

        return previsao

    def impressao(self, previsao):
        resultado = pd.DataFrame({'id': self.data_previsao['id'], 'Price_Prediction': previsao})
        resultado.to_csv('/home/caio/Estudo/kaggle_02/submission.csv', index=False)
        print("Previsão salva com sucesso!")


# Instanciando a classe Aurora
teste = Aurora(data_treino, data_prev)


Average MAE score:
34.11836874399781
Previsão salva com sucesso!
Aurora Finalizada Com Sucesso!
